# Check out how well a model is performing on different data

In this notebook several different models are compared with each other. This fitted model is then evaluated.


Also includes basic version of AR ARIMA and GLM models. 

In [ ]:
# import packages
import pandas as pd
import importlib
from datetime import datetime, timedelta
# import custom modules
from ts_forecasting import speccing, modelling, featuring
# useful for reloading after changes were made to the code
importlib.reload(speccing)
importlib.reload(modelling)
importlib.reload(featuring)
%matplotlib inline
PATH_TO_DATA = '../data/pickles'
modelling.MODEL_DIR = "../models"


def day_lags(lags):
    return  [l * 96 for l in lags]


def hour_lags(lags):
    return  [l * 4 for l in lags]

In [ ]:

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib

from pylab import rcParams
from collections import OrderedDict
plt.style.use('ggplot')
figsize = OrderedDict((('width', 14), ('height', 5)))
rcParams['figure.figsize'] = figsize.values()
rcParams['font.size'] = 15
rcParams['xtick.labelsize'] = 13
rcParams['ytick.labelsize'] = 13
rcParams['legend.frameon'] = True
rcParams['legend.framealpha'] = 0.8
rcParams['legend.fontsize'] = 13

In [ ]:
start = datetime(2015, 4, 13, 1, 45)

# Solar production
## With total radiation

In [ ]:

def create_difference(df: pd.DataFrame) -> pd.DataFrame:
    """
    create differnce between two variables, and drops these
    note: the variables used for the generation of the difference are dropped. So even if in ModelSpecs they are defined with the 'lags' parameter.
    """
    df['difference'] = df.solar_production_l24 - df.solar_production_l120
    df.drop(columns=['solar_production_l24','solar_production_l120'], inplace=True)
    return df


__The next three models are an investigation into the usefulness of adding very recent data (up to 6 hours ago), for the prediction of horizon of 6 hours.__

For comparison reasons, these models take into account only the last day (24 hours ago) and optionally a variable for the production of 6 hours ago.

In [ ]:
# Describe the model

## model uses data for production of 24 hours ago and the difference of 6 and 30 hours ago. The rationale for this is that the difference can help to improve predictions during the day:
## If you know at 9 AM that the production for that moment is higher or lower than 9 AM 24 hours ago, this can be useful for the prediction of 3 PM. 

outcome_var_spec = speccing.DFFileSeriesSpecs("%s/df_ss_pv_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="solar_production")
regressor_specs = [speccing.SeriesSpecs(column='y', name='difference')]
specs = modelling.ModelSpecs(outcome_var=outcome_var_spec,
                             model_type="OLS",
                             lags=hour_lags([6,24,30]),
                             regressors=regressor_specs,
                             start_of_data=start,
                             end_of_data=start + timedelta(days=32),
                             ratio_training_test_data=13 / 15,
                             horizons=["6H", "48H"],
                             transform=create_difference)
print(specs)



# Create and train the model
model = modelling.create_model(specs, "0.1")

# Evaluate the model
model.summary()
modelling.evaluate_models(m1=(model, specs))

In [ ]:
## statistics show that using the difference in this way is insigficant. Also the effect is very small (as can be seen from the graph above)

model.summary()

In [ ]:
# Describe the model

## model uses data for production of 24 hours ago and 6 hours ago.

outcome_var_spec = speccing.DFFileSeriesSpecs("%s/df_ss_pv_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="solar_production")
regressor_specs = [speccing.DFFileSeriesSpecs("%s/df_total_radiation_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="radiation_forecast_2days_l2")]
specs = modelling.ModelSpecs(outcome_var=outcome_var_spec,
                             model_type="OLS",
                             lags=hour_lags([6,24]),
                             regressors=[],
                             start_of_data=start,
                             end_of_data=start + timedelta(days=32),
                             ratio_training_test_data=13 / 15,
                             horizons=["6H", "48H"])
print(specs)

# Create and train the model
model = modelling.create_model(specs, "0.1")

# Evaluate the model
model.summary()
modelling.evaluate_models(m1=(model, specs))

In [ ]:
## small difference compared to previous model. slightly better RMSE and significant coef for solar_production_l24 and solar_production_l96

model.summary()

In [ ]:
# Describe the model
outcome_var_spec = speccing.DFFileSeriesSpecs("%s/df_ss_pv_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="solar_production")
regressor_specs = [speccing.DFFileSeriesSpecs("%s/df_total_radiation_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="radiation_forecast_2days_l2")]
specs = modelling.ModelSpecs(outcome_var=outcome_var_spec,
                             model_type="OLS",
                             lags=hour_lags([24]),
                             regressors=[],
                             start_of_data=start,
                             end_of_data=start + timedelta(days=32),
                             ratio_training_test_data=13 / 15,
                             horizons=["6H", "48H"])
print(specs)

# Create and train the model
model = modelling.create_model(specs, "0.1")

# Evaluate the model
model.summary()
modelling.evaluate_models(m1=(model, specs))

In [ ]:
## also small difference. Worst RMSE of three models

model.summary()

## conclusion

The three models compare almost the same. As discussed, data from 6 hours ago likely contains information that could improve the prediction with a 6 hour horizon over using data from 24 hours ago or longer. 

However, the current models above cannot make the best use of this information (models only slightly improve when data from 6 hours ago is added). One suggestion that is worth exploring is to see wether the '6 hour ago' information is only used when it is relevant. This can be done by adding a dummy variable that is '1' for 6 hours after the first radiation/production > 0 until the moment of the last radiation/production > 0 for that day. By creating an interaction term with this dummy, the 6 hour ago data is only used when it can most likely inprove the predictions. 

___
## example of generating a model with confidence interval or prediction interval for each prediction


In [ ]:
# Describe the model
outcome_var_spec = speccing.DFFileSeriesSpecs("%s/df_ss_pv_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="solar_production")
regressor_specs = [speccing.DFFileSeriesSpecs("%s/df_total_radiation_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="radiation_forecast_2days_l2")]
specs = modelling.ModelSpecs(outcome_var=outcome_var_spec,
                             model_type="OLS",
                             lags=hour_lags([6, 24]),
                             regressors=[],
                             start_of_data=start,
                             end_of_data=start + timedelta(days=32),
                             ratio_training_test_data=13 / 15,
                             horizons=["6H", "48H"])
print(specs)

# Create and train the model
model = modelling.create_model(specs, "0.1")

# Evaluate the model
model.summary()
modelling.evaluate_models(m1=(model, specs))

In [ ]:
# model.summary()

In [ ]:
predictions = model.get_prediction()
confidence_intervals = predictions.summary_frame()[['mean_ci_lower','mean_ci_upper']]
confidence_intervals['predictions'] = model.predict()
confidence_intervals.iloc[:200].plot()

In [ ]:
prediction_intervals = predictions.summary_frame()[['obs_ci_lower','obs_ci_upper']]
prediction_intervals['predictions'] = model.predict()
prediction_intervals.iloc[:200].plot()
## prediction intervals can be below zero :(

## arima
some time series models are compared below: OLS on panel data, an AR and an ARIMA model. Only the data frame is constructed using the modules. The actual models are run in the notebook by using the respective packages.

In [ ]:
# Describe the model
outcome_var_spec = speccing.DFFileSeriesSpecs("%s/df_ss_pv_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="solar_production")
regressor_specs = [speccing.DFFileSeriesSpecs("%s/df_total_radiation_res15T.pickle" % PATH_TO_DATA,
                                            column="y", name="radiation_forecast_2days_l2")]
specs = modelling.ModelSpecs(outcome_var=outcome_var_spec,
                             model_type="OLS",
                             lags=hour_lags([24,48]),
                             regressors=regressor_specs,
                             start_of_data=start,
                             end_of_data=start + timedelta(days=200),
                             ratio_training_test_data=13 / 15,
                             horizons=["6H", "48H"])
print(specs)


In [ ]:

## run on same hours accross days, so one lage will be the same time the day before for both the OLS model and AR models.

input_df = featuring.construct_dataframe(specs, 'train')
input_df_one_moment = input_df[(input_df.index.hour==15) & (input_df.index.minute==15)]
input_df_one_moment['const'] = 1
input_df_one_moment = input_df_one_moment[~pd.isnull(input_df_one_moment).any(axis=1)]


In [ ]:
input_df_one_moment.head()

In [ ]:
import statsmodels.api as sm
import numpy as np

In [ ]:
model = sm.OLS(np.array(input_df_one_moment.iloc[2:]['solar_production']),
                            input_df_one_moment.iloc[2:][['solar_production_l96','solar_production_l192','const']])

results = model.fit()

results.summary()

In [ ]:
# This produce the same results. Note that the first two observations are left out for OLS to make sure that exactly the same data is used for training: the first two observations have lagged variables, but AR cannot lag these observations. 
model = sm.tsa.AR(pd.DataFrame(input_df_one_moment['solar_production']))

results = model.fit(maxlag=2)
results.params


In [ ]:
# ARIMA model where order can be adjusted and exog variables can optionally be added.
model = sm.tsa.ARIMA(pd.DataFrame(input_df_one_moment['solar_production']), order=(2, 0, 2),
                    exog=pd.DataFrame(input_df_one_moment['radiation_forecast_2days_l2']))  

results = model.fit(maxlag=2)
results.params
results.summary()

## glm
At last a glm model is created that might help to overcome the <0 prediction intervals. This is still work in progress.

In [ ]:
# Instantiate a gamma family model with the default link function.
gamma_model = sm.GLM(np.array(input_df_one_moment.iloc[2:]['solar_production']),
                     input_df_one_moment.iloc[2:][['solar_production_l96','solar_production_l192','const']],
                     family=sm.families.Gamma())

gamma_results = gamma_model.fit()

gamma_results.summary()

In [ ]:
predictions = gamma_results.get_prediction()
confidence_intervals = predictions.summary_frame()[['mean_ci_lower','mean_ci_upper']]
confidence_intervals.plot()